In [68]:
import os
import importlib
import re, array, csv, datetime, glob, json, math, random, stat
import pytz, datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# This is a utility function for running other ipython notebooks
def exec_ipynb(filename_or_url):
    nb = (requests.get(filename_or_url).json() if re.match(r'https?:', filename_or_url) else json.load(open(filename_or_url)))
    if(nb['nbformat'] >= 4):
        src = [''.join(cell['source']) for cell in nb['cells'] if cell['cell_type'] == 'code']
    else:
        src = [''.join(cell['input']) for cell in nb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']
    tmpname = '/tmp/%s-%s-%d.py' % (os.path.basename(filename_or_url),
                                    datetime.datetime.now().strftime('%Y%m%d%H%M%S%f'),
                                    os.getpid())
    src = '\n\n\n'.join(src)
    open(tmpname, 'w').write(src)
    code = compile(src, tmpname, 'exec')
    exec(code, globals())


# Load utility functions from another ipython notebook
root_dir = "/projects/9ab71616-fcde-4524-bf8f-7953c669ebbb/air-src/"
os.chdir(root_dir + "linRegModel/pardump_example/")
exec_ipynb("pardumpdump-randy-amy-util.ipynb")
os.chdir(root_dir + "linRegModel")
exec_ipynb("./cachedHysplitRunLib.ipynb")
exec_ipynb("../../src/python-utils/utils.ipynb")

# Load EarthTime document template
os.chdir(root_dir + "automate-plume-viz/")
df_template = pd.read_csv("data/earth_time_template.csv")

# Create rows in the EarthTime document
start_d = pd.date_range(start="2019-01-01", end="2020-01-01", closed="left")
end_d = pd.date_range(start="2019-01-01", end="2020-01-01", closed="right")
start_d_str = start_d.strftime("%Y%m%d0200")
end_d_str = end_d.strftime("%Y%m%d0001")
df = df_repeated = pd.concat([df_template]*len(start_d_str), ignore_index=True)
file_name = "plume_" + start_d.strftime("%Y%m%d")
df["Start date"] = start_d_str
df["End date"] = end_d_str
df["Share link identifier"] = file_name
df["Name"] = start_d.strftime("%Y-%m-%d")
df["URL"] = "https://cocalc-www.createlab.org/pardumps/" + file_name + ".bin"

# Save rows
p = "data/earth_time.csv"
df.to_csv(p)
os.chmod(p, 0o777)